## 使用keras训练notMNIST数据集，尝试之前作业的两种CNN

- 由于keras和tensorflow版本的问题，这里通过tensorflow调用keras，而不是直接import keras

In [11]:
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Dense, Dropout, Activation,Convolution2D, ZeroPadding2D, MaxPooling2D,Flatten
from tensorflow.keras.optimizers import SGD  
from tensorflow.keras.callbacks import Callback
import numpy as np
from sklearn.metrics import roc_auc_score

In [1]:
import pickle
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [2]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            accur = accuracy(y_pred,self.y_val)
            print("\n ROC-AUC - epoch: %d - score: %.6f - accuracy:  %.1f%% \n" % (epoch+1, score,accur))

In [6]:
RocAuc = RocAucEvaluation(validation_data=(valid_dataset, valid_labels), interval=1)

- CNN without pooling

In [12]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

model = Sequential()
model.add(Convolution2D(depth, (patch_size,patch_size),padding = 'SAME',strides=(2,2),input_shape=(28, 28,1),activation='relu'))
model.add(Convolution2D(depth, (patch_size, patch_size),padding = 'SAME',strides=(2,2),activation='relu'))
model.add(Flatten())
model.add(Dense(num_hidden,activation='relu'))
model.add(Dense(10))
model.add(Activation('softmax'))
sgd = SGD(lr=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=sgd) 
model.fit(train_dataset,train_labels,batch_size=batch_size,epochs=5,callbacks=[RocAuc], verbose=2,
          validation_data=(valid_dataset, valid_labels))

Train on 200000 samples, validate on 10000 samples
Epoch 1/5

 ROC-AUC - epoch: 1 - score: 0.988411 - accuracy:  87.0% 

200000/200000 - 37s - loss: 0.5659 - val_loss: 0.4272
Epoch 2/5

 ROC-AUC - epoch: 2 - score: 0.990856 - accuracy:  88.5% 

200000/200000 - 38s - loss: 0.3867 - val_loss: 0.3777
Epoch 3/5

 ROC-AUC - epoch: 3 - score: 0.991849 - accuracy:  89.2% 

200000/200000 - 39s - loss: 0.3461 - val_loss: 0.3585
Epoch 4/5

 ROC-AUC - epoch: 4 - score: 0.992361 - accuracy:  89.6% 

200000/200000 - 40s - loss: 0.3222 - val_loss: 0.3466
Epoch 5/5

 ROC-AUC - epoch: 5 - score: 0.993067 - accuracy:  90.2% 

200000/200000 - 40s - loss: 0.3060 - val_loss: 0.3281


- CNN with pooling

In [13]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

model = Sequential()
model.add(Convolution2D(depth, (patch_size,patch_size),padding = 'SAME',strides=(1,1),input_shape=(28, 28,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Convolution2D(depth, (patch_size, patch_size),padding = 'SAME',strides=(1,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(num_hidden,activation='relu'))
model.add(Dense(10))
model.add(Activation('softmax'))
sgd = SGD(lr=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=sgd) 
model.fit(train_dataset,train_labels,batch_size=batch_size,epochs=5,callbacks=[RocAuc], verbose=2,
          validation_data=(valid_dataset, valid_labels))

Train on 200000 samples, validate on 10000 samples
Epoch 1/5

 ROC-AUC - epoch: 1 - score: 0.990325 - accuracy:  88.6% 

200000/200000 - 78s - loss: 0.5035 - val_loss: 0.3895
Epoch 2/5

 ROC-AUC - epoch: 2 - score: 0.992107 - accuracy:  89.5% 

200000/200000 - 80s - loss: 0.3520 - val_loss: 0.3520
Epoch 3/5

 ROC-AUC - epoch: 3 - score: 0.993240 - accuracy:  90.3% 

200000/200000 - 81s - loss: 0.3182 - val_loss: 0.3261
Epoch 4/5

 ROC-AUC - epoch: 4 - score: 0.993661 - accuracy:  90.6% 

200000/200000 - 80s - loss: 0.2972 - val_loss: 0.3151
Epoch 5/5

 ROC-AUC - epoch: 5 - score: 0.994009 - accuracy:  90.7% 

200000/200000 - 81s - loss: 0.2825 - val_loss: 0.3081
